Hi

In this section of trading and financial, I'm gonna cover optimization.
Investors always looking for a way to gain maximum return and minimum loss, so we use the Optimization method to do it.

In [32]:
import pandas as pd
import numpy as np
import datetime as dt
from pylab import mpl, plt
plt.style.use('seaborn')

First of all i import "itertool" and product from that .

In [2]:
from itertools import product

Now I read the data that was prepared from pre section.

In [16]:
df = pd.read_csv('df.csv', index_col = 0, parse_dates = True)


In [9]:
data = pd.read_csv('data.csv', index_col = 0, parse_dates = True)
data.head()

,AMZN.O,SMA_250,SMA_50,Position,Return,Strategy
Date,,,,,,
2010-12-30,182.75,138.999131,172.5002,1,-0.003387,-0.003387
2010-12-31,180.00,139.180371,172.9268,1,-0.015162,-0.015162
2011-01-03,184.22,139.388251,173.3118,1,0.023174,0.023174
2011-01-04,185.01,139.608291,173.6294,1,0.004279,0.004279
2011-01-05,187.42,139.823891,173.9978,1,0.012942,0.012942


### Optimization

In [17]:
symbol = 'AMZN.O'

In [18]:
sma1 = range(20,61,4)
sma2 = range(180,281,10)

results = pd.DataFrame()
for SMA1,SMA2 in product(sma1, sma2):
    data = pd.DataFrame(df[symbol])
    data.dropna(inplace = True)
    data['Returns'] = np.log(data[symbol]/data[symbol].shift(1))
    data['SMA1'] = data[symbol].rolling(SMA1).mean()
    data['SMA2'] = data[symbol].rolling(SMA2).mean()
    
    data.dropna(inplace = True)
    data['Possition'] = np.where(data['SMA1'] > data['SMA2'],1,-1)
    data['Strategy'] = data['Possition'].shift(1) * data['Returns'] 
    data.dropna(inplace = True)
    perf = np.exp(data[['Returns','Strategy']].sum())
    results = results.append(pd.DataFrame({
        'SMA1':SMA1, 'SMA2':SMA2,
        'MARKET':perf['Returns'],
        'Strategy':perf['Strategy'],
        'OUT' : perf['Strategy']- perf['Returns']},
        index = [0]), ignore_index = True)
        
    

In [19]:
results.sort_values('OUT',ascending=False).head(7)

,SMA1,SMA2,MARKET,Strategy,OUT
5,20,230,9.690992,5.533115,-4.157877
14,24,210,10.455161,5.912703,-4.542457
119,60,270,9.215506,4.603016,-4.612489
120,60,280,9.128403,4.458517,-4.669886
118,60,260,9.234029,4.378271,-4.855758
3,20,210,10.455161,5.582577,-4.872583
16,24,230,9.690992,4.765967,-4.925025


As we see in table abow, according to brut-force optimization, SMA1 = 20 and SMA2 = 230 are the best parameters to maximize the OUT value.

This results are heavily dependent on the dataset that we use.

### Random Walk Hypathesis

The random walk theory suggests that changes in stock prices have the same distribution and are independent of each other, therefore, the past movement or trend of a stock price or market cannot be used to predict its future movement. In short, this is the idea that stocks take a random and unpredictable path.

A proponent of the random walk theory believes it's impossible to outperform the market without assuming additional risk. Critics of the theory, however, contend that stocks do maintain price trends over time – in other words, that it is possible to outperform the market by carefully selecting entry and exit points for equity investments.

Using python, the RWH is tested here as follow:financial time series of historical market prices is used so number of lags are created, also Regression will use to prediction.

The main idea is yesterday and other 4 more days back can be used to predict todays market prices.

In [20]:
sym = '.SPX'

data = pd.DataFrame(df[sym])

lags = 5
cols =[]
for lag in range(1, lags + 1):
    col = 'lag_{}'.format(lag)
    data[col] = data[sym].shift(lag)
    cols.append(col)
    
    

In [22]:
data.head(8)

,.SPX,lag_1,lag_2,lag_3,lag_4,lag_5
Date,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,1132.99,NaN,NaN,NaN,NaN,NaN
2010-01-05,1136.52,1132.99,NaN,NaN,NaN,NaN
2010-01-06,1137.14,1136.52,1132.99,NaN,NaN,NaN
2010-01-07,1141.69,1137.14,1136.52,1132.99,NaN,NaN
2010-01-08,1144.98,1141.69,1137.14,1136.52,1132.99,NaN
2010-01-11,1146.98,1144.98,1141.69,1137.14,1136.52,1132.99
2010-01-12,1136.22,1146.98,1144.98,1141.69,1137.14,1136.52


In [23]:
data.dropna(inplace = True)